Multi-Class Prediction of Obesity Risk

https://www.kaggle.com/competitions/playground-series-s4e2/overview

v1: 随机森林
score: 0.89631   ranking: 68/86 

anthor: moonzfxs
time: 2024/02/01

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")
print(train_df.shape)
train_df.head()

(20758, 18)


,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
print(train_df['NObeyesdad'].unique())

['Overweight_Level_II' 'Normal_Weight' 'Insufficient_Weight'
 'Obesity_Type_III' 'Obesity_Type_II' 'Overweight_Level_I'
 'Obesity_Type_I']


In [4]:
train_target = train_df['NObeyesdad']
train_df = train_df.drop(columns=['NObeyesdad'])
train_df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation


In [5]:
unique_values, counts = np.unique(train_target, return_counts=True)

# 打印每个唯一数及其出现次数
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

Insufficient_Weight: 2523
Normal_Weight: 3082
Obesity_Type_I: 2910
Obesity_Type_II: 3248
Obesity_Type_III: 4046
Overweight_Level_I: 2427
Overweight_Level_II: 2522


In [6]:
from sklearn.preprocessing import LabelEncoder

# 初始化LabelEncoder
label_encoder = LabelEncoder()

# 将字符串列转换为数字列
train_target = label_encoder.fit_transform(train_target)
print(np.unique(train_target))

[0 1 2 3 4 5 6]


In [7]:
unique_values, counts = np.unique(train_target, return_counts=True)

# 打印每个唯一数及其出现次数
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0: 2523
1: 3082
2: 2910
3: 3248
4: 4046
5: 2427
6: 2522


In [8]:
from sklearn.preprocessing import StandardScaler

def feature_processing(data):
    data = data.drop(columns=['id'])
    data = data.apply(lambda col: label_encoder.fit_transform(col) if col.dtype == 'object' else col)
    print(type(data))
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    scaled_data = pd.DataFrame(scaled_data, columns = data.columns)
    return scaled_data

In [9]:
train_df = feature_processing(train_df)
train_df.head()

<class 'pandas.core.frame.DataFrame'>


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1.004152,0.105699,-0.002828,-0.235713,0.469099,0.30588,-0.836279,0.314684,0.337845,-0.109287,1.206594,-0.185009,-1.171141,0.597438,-0.471288,0.429319
1,-0.995866,-1.027052,-1.606291,-1.170931,0.469099,0.30588,-0.836279,0.338364,-1.889204,-0.109287,-0.048349,-0.185009,0.021775,0.636513,1.639846,-2.182324
2,-0.995866,-1.027052,0.128451,-1.430012,0.469099,0.30588,-1.060332,-1.913423,0.337845,-0.109287,-0.195644,-0.185009,-0.138022,1.755239,1.639846,0.429319
3,-0.995866,-0.507929,0.120090,1.644770,0.469099,0.30588,1.039171,0.338364,0.337845,-0.109287,-0.584035,-0.185009,0.579896,0.271455,-0.471288,0.429319
4,1.004152,1.371197,2.450367,0.224054,0.469099,0.30588,0.438397,-1.119801,0.337845,-0.109287,-0.081469,-0.185009,1.176486,0.523111,-0.471288,0.429319


In [10]:
from sklearn.model_selection import train_test_split

print(train_df.shape)
print(np.size(train_target))
X_train, X_test, y_train, y_test = train_test_split(train_df, train_target, test_size=0.4, random_state=42)

(20758, 16)
20758


In [11]:
# from sklearn.linear_model import LinearRegression 

# model = LinearRegression()
# model.fit(X_train, y_train)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)


# 5. 模型预测
y_pred = rf_classifier.predict(X_test)

# 6. 模型评估
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.895592485549133


In [13]:
test_ids = test_df['id']

test_df = feature_processing(test_df)  # (110023, 12)

test_df.head()

<class 'pandas.core.frame.DataFrame'>


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1.006524,0.508163,1.682789,1.273786,0.46448,0.316064,0.932524,0.350808,0.341484,-0.114792,1.298389,-0.18625,-0.141769,-1.005016,-0.479492,0.437722
1,-0.993518,-0.509128,-1.114658,-0.818988,0.46448,0.316064,-0.833161,-2.462523,0.341484,-0.114792,1.583678,-0.18625,0.030307,-1.005016,-0.479492,0.437722
2,-0.993518,0.353000,-0.626191,0.927432,0.46448,0.316064,1.047997,0.350808,0.341484,-0.114792,0.965029,-0.18625,-1.159701,-0.592995,-0.479492,0.437722
3,1.006524,-0.512705,-1.642760,0.623672,0.46448,0.316064,-0.833161,0.319733,0.341484,-0.114792,1.234234,-0.18625,-1.046828,-1.005016,-0.479492,0.437722
4,-0.993518,0.353000,-0.805996,0.668336,0.46448,0.316064,1.047997,0.350808,0.341484,-0.114792,1.016818,-0.18625,-1.159701,0.213881,-0.479492,0.437722


In [14]:
pred = rf_classifier.predict(test_df)
# sub_target = sub_target.round()
print(np.unique(pred))

[0 1 2 3 4 5 6]


In [15]:
# sub_target = label_encoder.inverse_transform(pred)

mapping = {
    0: 'Insufficient_Weight',
    1: 'Normal_Weight',
    2: 'Obesity_Type_I',
    3: 'Obesity_Type_II',
    4: 'Obesity_Type_III',
    5: 'Overweight_Level_I',
    6: 'Overweight_Level_II'
}

# 使用列表推导式将数组中的数字映射为字符串
sub_target = np.array([mapping[num] for num in pred])

print(sub_target)

['Obesity_Type_II' 'Overweight_Level_I' 'Obesity_Type_III' ...
 'Insufficient_Weight' 'Normal_Weight' 'Obesity_Type_II']


In [16]:
sub = pd.DataFrame({'id': test_ids, 'NObeyesdad': sub_target})

sub.to_csv('submission.csv', index=False)

In [17]:
print(sub)

          id           NObeyesdad
0      20758      Obesity_Type_II
1      20759   Overweight_Level_I
2      20760     Obesity_Type_III
3      20761       Obesity_Type_I
4      20762     Obesity_Type_III
...      ...                  ...
13835  34593  Overweight_Level_II
13836  34594   Overweight_Level_I
13837  34595  Insufficient_Weight
13838  34596        Normal_Weight
13839  34597      Obesity_Type_II

[13840 rows x 2 columns]
